Run tensor cell2cell on outputs of various communication scoring tools

env name: cci_dt


In [1]:
import os
from collections import OrderedDict
import itertools
import numpy as np
import pandas as pd

import cell2cell as c2c

seed = 888
np.random.seed(seed)

rev_path = '/data3/hratch/tc2c_analyses_1/natcomm_revisions/'
expression_data_path = '/data2/hratch/immune_CCI/covid/expression_data/covid_data/'

cp_delim = '-'
lr_delim = '&'

In [2]:
import sys
sys.path.insert(1, '/home/hratch/Projects/CCC/tc2c_analyses_1/notebooks/natcomm_revisions/')
from utility import matrix_to_interaction_tensor

Load output of the different communication scoring pipelines

In [6]:
samples = ['C51', 'C52', 'C100', 'C141', 'C142',  'C144',
 'C145', 'C143', 'C146', 'C148', 'C149', 'C152']

In [7]:
# load cellchat
cellchat_cm = pd.read_csv(rev_path + 'interim/tc2c_external_inputs/cellchat/cellchat_balf.csv', index_col = 0)
cellchat = OrderedDict()
for sample in samples:
    df = cellchat_cm[cellchat_cm.Sample == sample]
    df.drop(columns = ['Sample'], inplace = True)
    df.index = pd.Series(df.index).apply(lambda x: x.split('.')[1]).tolist()
    cellchat[sample] = df

In [8]:
# load NATMI
natmi_out_path = rev_path + 'interim/tc2c_external_inputs/natmi/'
natmi = OrderedDict()
for sample in samples:
    natmi[sample] = pd.read_csv(natmi_out_path + sample + '_communication_matrix.csv', index_col = 0)


In [120]:
cellchat_tensor = matrix_to_interaction_tensor(scores = cellchat, cell_delim='-', lr_delim='&', 
                                              lr_how='outer', lr_fill=float('nan'), 
                                              cell_how='outer', cell_fill=0, 
                                              prioritize_lr_fill=False)

natmi_tensor = matrix_to_interaction_tensor(scores = natmi, cell_delim='-', lr_delim='&', 
                                              lr_how='outer', lr_fill=float('nan'), 
                                              cell_how='outer', cell_fill=0, 
                                              prioritize_lr_fill=False)

In [14]:
cellchat_tensor.tensor.shape

(6, 6, 446, 12)

In [15]:
natmi_tensor_inner.tensor.shape

(6, 6, 446, 12)

In [ ]:
# make sure all tensors are same shape and in same order

Transform into 4D Interaction Tensor
Make all communication matrices consistent -- contain all same LR and CC pairs in the same order

In [26]:
# # get all possible cell type pairs -- union of all LR pairs and CC pairs measured by all methods

# # only necessary bc multiple pipelines, otherwise can feed directly into "matrix_to_communication_tensor" with how = 'outer'
# score_methods = [cellchat, ]
# method_names = ['cellchat', ]


# cell_pairs = []
# lr_pairs = []
# for score_method in score_methods:
#     cp = [df.columns.tolist() for df in score_method.values()]
#     cp = [item for sublist in cp for item in sublist]
#     cell_pairs += cp
    
#     lr = [df.index.tolist() for df in score_method.values()]
#     lr = [item for sublist in lr for item in sublist]
#     lr_pairs += lr

In [27]:
# cell_pairs = sorted(set(cell_pairs))
# lr_pairs = sorted(set(lr_pairs))



In [32]:
# def create_nan_vals_all(df, lr_pairs, cell_pairs):
#     """Adds nan communication scores if pipeline did not include a certain sender-receiver cell pair"""
#     df = pd.concat([df, pd.DataFrame(index = set(lr_pairs).difference(df.index),columns = df.columns)])
#     for col in set(cell_pairs).difference(df.columns):
#         df[col] = float('nan')
    
#     df = df.loc[lr_pairs, cell_pairs] # make the order the same
#     return df

# tensors = {}
# for idx, score_method in enumerate(score_methods):
#     print(method_names[idx])
#     scores = OrderedDict({k: create_nan_vals_all(df=score_method[k], lr_pairs=lr_pairs, cell_pairs=cell_pairs) for k in samples})
#     score_methods[idx] = scores
#     tensors[method_names[idx]] = matrix_to_interaction_tensor(scores, how = 'outer', 
#                                                              cp_delim='-', lr_delim='&')

  0%|          | 0/12 [00:00<?, ?it/s]

cellchat


100%|██████████| 12/12 [00:24<00:00,  2.05s/it]
